# 패키지 설치하기

In [1]:
!pip install ratsnlp

     |████████████████████████████████| 42 kB 221 kB/s 
     |████████████████████████████████| 806 kB 8.7 MB/s 
     |████████████████████████████████| 57 kB 4.0 MB/s 
     |████████████████████████████████| 2.8 MB 28.7 MB/s 
     |████████████████████████████████| 829 kB 37.4 MB/s 
     |████████████████████████████████| 408 kB 34.1 MB/s 
     |████████████████████████████████| 136 kB 35.8 MB/s 
     |████████████████████████████████| 636 kB 47.3 MB/s 
     |████████████████████████████████| 77 kB 6.6 MB/s 
     |████████████████████████████████| 895 kB 43.8 MB/s 
     |████████████████████████████████| 3.3 MB 33.7 MB/s 
     |████████████████████████████████| 1.1 MB 41.8 MB/s 
     |████████████████████████████████| 96 kB 5.8 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
     |████████████████████████████████| 144 kB 36.3 MB/s 
     |████████████████████████████████| 271 kB 33.7 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha

# 환경 설정

모델 hyperparameter와 저장 위치 등 설정 정보를 선언

In [2]:
from ratsnlp.nlpbook.qa import QADeployArguments

ImportError: ignored

In [ ]:
args = QADeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/content/drive/Othercomputers/내 컴퓨터/Chapter 7. Question Answering/checkpoint-qa",
    max_seq_length=128,
    max_query_length=32,
)

# 모델 불러오기

파인튜닝을 마친 모델 불러오기

In [ ]:
import torch
from transformers import BertConfig, BertForQuestionAnswering

In [ ]:
# 체크포인트 로드
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu")
)

# BERT 설정 로드
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)

model = BertForQuestionAnswering(pretrained_model_config)      # 모델 초기화
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})     # 체크포인트 주입
model.eval()    # 평가모드

토크나이저 준비

In [ ]:
from transformers import BertTokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

# 인퍼런스 함수

In [ ]:
def inference_fn(question, context):
    if question and context:

        # 질문을 토큰화, 길이가 길면 자르기
        truncated_query = tokenizer.encode(
            question,
            add_special_tokens=False,
            truncation=True,
            max_length=args.max_query_length
        )

        # 앞서 처리한 질문을 지문과 함께 토큰화, 길이가 길면 자르기
        inputs = tokenizer.encode_plus(
            text=truncated_query,
            text_pair=context,
            truncation="only_second",
            padding="max_length",
            max_length=args.max_seq_length,
            return_token_type_ids=True,
        )

        with torch.no_grad():
            outputs = model(**{k: torch.tensor([v]) for k, v in inputs.items()})    # 토큰화 된 데이터를 텐서로 변환 후 모델에 입력
            start_pred = outputs.start_logits.argmax(dim=-1).item()     # 정답의 시작 위치와 관련 된 로짓에서 가장 큰 값을 가진 토큰 찾기
            end_pred = outputs.end_logits.argmax(dim=-1).item()         # 정답의 끝 위치와 관련 된 로짓에서 가장 큰 값을 가진 토큰 찾기
            pred_text = tokenizer.decode(inputs['input_ids'][start_pred:end_pred+1])        # 정답 시작토큰 부터 끝토큰 이어붙이기
    else:
        pred_text = ""
    return {
        'question': question,
        'context': context,
        'answer': pred_text,
    }

# 웹서비스 개시

`ngrok` 은 코랩 로컬에서 실행 중인 웹서비스를 안전하게 외부에서 접근 가능하도록 해주는 도구이다. 실행하려면 [회원가입](https://dashboard.ngrok.com/signup) 후 [로그인](https://dashboard.ngrok.com/login)을 한 뒤 [이곳](https://dashboard.ngrok.com/get-started/your-authtoken)에 접속해 인증 토큰(authtoken)을 입력해야 한다.

In [ ]:
!mkdir /root/.ngrok2 && echo "authtoken: 27QQLX2ewrzfpeSLcvUEBNkqKaE_5zhvxzJsNdMkffKpAkuZE" > /root/.ngrok2/ngrok.yml

인퍼런스 함수를 Flask를 통해 웹서비스로 만든다.

In [ ]:
from ratsnlp.nlpbook.paircls import get_web_service_app

In [ ]:
app = get_web_service_app(inference_fn)
app.run()